**<center><h1>KERAS.LOSSES</h1></center>**<br>
Dans keras.losses, on a des fonctions et des classes:<br>
1. **keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=True)** est une fonction qu'on directement utilisée dans tf.GradientTape.<br>
2. **a = keras.losses.CategoricalCrossentropy(from_logits=True)** est une classe, on fait **a(y_true, y_pred)**.<br>
On a aussi **KERAS.METRICS**, **KERAS.OPTIMIZERS**, ETC.<br>

**<center><h1>Training Keras Model</h1></center>**<br>
On peut noter **04** approches:<br>
1. **fit:** c'est la méthode standard.<br>

2. **keras.Model:** Créer une classe qui hérite de la classe **keras.models.Model** or **keras.Model**. Cette classe doit implémenter **train_step** qui utilise **tf.GradientTape** pour mettre à jour les **poids** du modèle par **batch**. On peut aussi implémenter **test_step** pour le test, **call** pour le **forward**, **metrics** qui retourne la liste des métriques (mise à jour), **computer_loss** pour retourner la **loss**.<br>

3. **@tf.function:** Créer une fonction en la faisant précédée par **@tf.function**. Cette approche permet à la fonction créée de pouvoir être compiler c'est à dire utiliser le **tf.GradientTape**. La fonction ainsi créée peut être utilisée dans une fonction qui fait une boucle sur le nombre d'époques et donne entrée **batch** par **batch** à la fonction qui met à jour les poids suivant les batchs donnés.<br>

4. **train_on_batch:** C'est une **méthode** de la classe Model de keras, tout modèle de keras peut donc y accéder. On crée d'abord un modèle **model**, on utilise d'abord une boucle sur le nombre d'époque suivie d'une boucle qui prend **batch par batch**, on fait à chaque fois **model.train_on_batch(x_batch, y_batch)** qui retourne la **loss** sur le batch et les métriques si c'es définit dans le compile de **model**.<br>

**<center><h1>Passage par valeur des modèles</h1></center>**<br>
Supposons qu'on ait 03 modèles: <br>

**model1 = tf.keras.models.Sequential()**<br>
**model2 = tf.keras.models.Sequential()**<br>
**model3 = tf.keras.models.Sequential()**<br>

**model3.add(model1**)<br>
**mdoel3.model1.trainable = False** <br>
**model3.add(model2)**<br>

model1 est utilisé dans model3, même si c'est son trainable=False, toute modification (mise à jour) fait sur model1 en dehors va se répercuter sur model3.model1, même si les modifications sont faites à l'intérieur d'une fonction.<br>
**NB:** Dès qu'on crée un modèle, on l'utilise pour intialiser ou l'affecter ou à un autre modèle les modifications apportées sur un de ces modèles affectent le modèle de base. Voir la section: Method using train_on_batch.<br>

**<center><h1>Bahdanau Attention</h1></center>**<br>
C'est une attention additive, l'objectif est de prendre l'état caché du decoder comme query et le output de l'encoder comme value pour produre un vecteur de contexte. Le vecteur de contexte est concaténé avec l'embedding de l'entrée du decoder (qui prend token par token), et sert d'entrée à la couche rnn, lstm ou gru.<br>

Q = Decoder hidden state (None, decoder_unit)<br>
    at the begining, Q = Encoder hidden state<br>

V = Encoder output (None, sequence, feature)<br>

Q = tf.expand_dims(Q, axis=1) # (None, 1, decoder_unit)<br>
EncQ = Dense1(Q) # (None, units)<br>
EncV = Dense2(V) # (None, sequence, units)<br>
sum = tf.nn.tanh(EncQ+EncV) # activation (None, sequence, units)<br>

score = Dense3(sum) # (None, sequence, 1)<br>
attention = tf.nn.softmax(score, axis=1) # (None, sequence, 1) softmax based on token position<br>

context_vector = attention*V # (None, sequence, feature)<br>
context_vector = tf.nn.sum(context_vector, axis=1) # (None, feature)<br>

context_vector sera utilisé dans le decoder et concaténer avec l'entrée (qui se fait token par token)<br>
x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) # (None, 1, feature+embedding_dim_x) x vient de l'embedding x est de type (None, 1) car on traite par token Embedding donne (None, 1, embedding_dim_x)<br>


**<center><h1>Paramètres et Sorties</h1></center>**<br>

**<h2>MLP (Dense)</h2>**<br>
**1. Sortie:** Si l'entrée est de dimension 2, la sortie sera **(None, units)**, si l'entrée est de dimension 3 (séquences), la sortie sera **(None, sequence, units)**, si l'entrée est de dimension 4 (convolutions ou images), la sortie sera **(None, H, W, units)**.<br>

**2. Paramètres:** **D*W + W**, avec D la dimension de l'entrée, W le nombre de units.<br>

**<h2>CNN (Conv2D)</h2>**<br>
**1. Sortie:** **(n - f + p) / s**, avec n la taille de l'entrée, f la taille du filtre, p = taille du filtre si padding = 'same' et p = 0 si padding='valid', s est la taille du stride. ON considère la partie entière inférieure.<br>
**NB:** Si conv2DTranspose ou UpSampling2D, on effectue l'opération inverse **s × (n + f - p)**.<br>


**Paramètres:** Le nombre de paramètres de la couche, dépend du nombre de canal de la couche précédente. La couche actuelle contient par exemple 2 filtres, et que la couche précédente contient 3 canaux, le filtre 1 va être dupliqué sur les 3 canaux et si la taille du filtre est (2, 2), donc y aura 3 × (2 × 2) paramètres à apprendre avec le même biais donc 3 × (2 × 2) + 1. Le même processus s'applique sur le filtre 2. La formule générale est alors:<br> <center>**(hauteur_filtre × largeur_filtre × nombre_canaux_couche_precedente + 1 ) × nombre_filtres**</center><br>

**Les couches de Pooling n'ont pas de paramètres à apprendre**<br>


**<h2>RNN</h2>**<br>
**1. Sortie:** Par défaut, retourne (None, units). Si return_sequences = True, (None, sequence, units). Si return_state = True, on aura deux outputs, (None, units), (None, units). Si return_sequences=True et return_state=True on aura (None, sequence, units), (None, units).<br>
**NB:** Ceci est valable pour les **GRU**, pour les **LSTM**, return_sate=True donne deux état (None, units) pour hidden_state et (None, units) pour cell_state.

**2. Paramètres:** L'entrée correspond à deux vecteurs concaténés: x (dernière dimension de l'entrée: **shape[-1]**) et h. x le token d'entrée et h l'état de caché.<br>
**<center>RNN: (x + h) × h + h</center>**<br>
**<center>LSTM: 4 × [(x + h) × h + h] </center>** <br>
**<center>GRU: 3 × [(x + h) × h + h]</center>**

# **Tensorflow Graph concept: GradientTape**

In [ ]:
"""
EXAMPLE : LOSS IS NOT GIVEN IN COMPILE
"""

class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [self.loss_tracker, self.mae_metric] ## List all metrics

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = keras.losses.mean_squared_error(y, y_pred) ## if loss if given in comile, we can use by self.compute_loss

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars)) ## optimizer is given in self.optimizer

        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()} ## bar progress for user {m.name:m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False) ## Freeze weights
        loss = keras.losses.mean_squared_error(y, y_pred)
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {m.name:m.result() for m in self.metrics}

# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

In [ ]:
"""
EXAMPLE : LOSS IS GIVEN IN COMPILE
"""

class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

    @property
    def metrics(self):
        # We list our `Metric` objects here so that `reset_states()` can be
        # called automatically at the start of each epoch
        # or at the start of `evaluate()`.
        # If you don't implement this property, you have to call
        # `reset_states()` yourself at the time of your choosing.
        return [self.loss_tracker, self.mae_metric] ## List all metrics

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute our own loss
            loss = self.compute_loss(y, y_pred) ### Changed line

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars)) ## optimizer is given in self.optimizer

        # Compute our own metrics
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {"loss": self.loss_tracker.result(), "mae": self.mae_metric.result()} ## bar progress for user {m.name:m.result() for m in self.metrics}

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False) ## Freeze weights
        loss = self.compute_loss(y, y_pred) ### Changed line
        self.loss_tracker.update_state(loss)
        self.mae_metric.update_state(y, y_pred)
        return {m.name:m.result() for m in self.metrics}

# Construct an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)

# We don't passs a loss or metrics here.
model.compile(optimizer="adam", loss="mse")

# Just use `fit` as usual -- you can use callbacks, etc.
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=5)

In [ ]:
"""
EXAMPLE: PONDERATION OF CLASSES USING WEIGHTS
"""

class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        if len(data) == 3:
            x, y, sample_weight = data
        else:
            sample_weight = None
            x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value.
            # The loss function is configured in `compile()`.
            loss = self.compute_loss(
                y=y,
                y_pred=y_pred,
                sample_weight=sample_weight,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics.
        # Metrics are configured in `compile()`.
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred, sample_weight=sample_weight)

        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}


# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# You can now use sample_weight argument
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
sw = np.random.random((1000, 1))
model.fit(x, y, sample_weight=sw, epochs=3)

# **Tensorflow Graph concept - GAN EXAMPLE: GradientTape**

In [ ]:
from tensorflow.keras import layers

# Create the discriminator
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)

# Create the generator
latent_dim = 128
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_tracker = keras.metrics.Mean(name="d_loss")
        self.g_loss_tracker = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        if isinstance(real_images, tuple):
            real_images = real_images[0]
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape: ##  We can either use with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics and return their value.
        self.d_loss_tracker.update_state(d_loss)
        self.g_loss_tracker.update_state(g_loss)
        return {
            "d_loss": self.d_loss_tracker.result(),
            "g_loss": self.g_loss_tracker.result(),
        }

In [ ]:
# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 64
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices(all_digits)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

# To limit the execution time, we only train on 100 batches. You can train on
# the entire dataset. You will need about 20 epochs to get nice results.
gan.fit(dataset.take(100), epochs=1)

# **Tensorflow Graph concept - Image and Text Similarity: GradientTape**

In [ ]:
"""
EXAMPLE: In this example, the loss is computing on the class
         the optimizer is given in compile, we use self.optimizer to use it.
         https://keras.io/examples/vision/nl_image_search/
"""
class DualEncoder(keras.Model):
    def __init__(self, text_encoder, image_encoder, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.text_encoder = text_encoder
        self.image_encoder = image_encoder
        self.temperature = temperature
        self.loss_tracker = keras.metrics.Mean(name="loss")

    @property
    def metrics(self):
        return [self.loss_tracker]

    def call(self, features, training=False):
        # Place each encoder on a separate GPU (if available).
        # TF will fallback on available devices if there are fewer than 2 GPUs.
        with tf.device("/gpu:0"):
            # Get the embeddings for the captions.
            caption_embeddings = text_encoder(features["caption"], training=training)
        with tf.device("/gpu:1"):
            # Get the embeddings for the images.
            image_embeddings = vision_encoder(features["image"], training=training)
        return caption_embeddings, image_embeddings

    def compute_loss(self, caption_embeddings, image_embeddings):
        # logits[i][j] is the dot_similarity(caption_i, image_j).
        logits = (
            tf.matmul(caption_embeddings, image_embeddings, transpose_b=True)
            / self.temperature
        )
        # images_similarity[i][j] is the dot_similarity(image_i, image_j).
        images_similarity = tf.matmul(
            image_embeddings, image_embeddings, transpose_b=True
        )
        # captions_similarity[i][j] is the dot_similarity(caption_i, caption_j).
        captions_similarity = tf.matmul(
            caption_embeddings, caption_embeddings, transpose_b=True
        )
        # targets[i][j] = avarage dot_similarity(caption_i, caption_j) and dot_similarity(image_i, image_j).
        targets = keras.activations.softmax(
            (captions_similarity + images_similarity) / (2 * self.temperature)
        )
        # Compute the loss for the captions using crossentropy
        captions_loss = keras.losses.categorical_crossentropy(
            y_true=targets, y_pred=logits, from_logits=True
        )
        # Compute the loss for the images using crossentropy
        images_loss = keras.losses.categorical_crossentropy(
            y_true=tf.transpose(targets), y_pred=tf.transpose(logits), from_logits=True
        )
        # Return the mean of the loss over the batch.
        return (captions_loss + images_loss) / 2

    def train_step(self, features):
        with tf.GradientTape() as tape:
            # Forward pass
            caption_embeddings, image_embeddings = self(features, training=True)
            loss = self.compute_loss(caption_embeddings, image_embeddings)
        # Backward pass
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Monitor loss
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, features):
        caption_embeddings, image_embeddings = self(features, training=False)
        loss = self.compute_loss(caption_embeddings, image_embeddings)
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

In [ ]:
num_epochs = 5  # In practice, train for at least 30 epochs
batch_size = 256

vision_encoder = create_vision_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)
text_encoder = create_text_encoder(
    num_projection_layers=1, projection_dims=256, dropout_rate=0.1
)
dual_encoder = DualEncoder(text_encoder, vision_encoder, temperature=0.05)
dual_encoder.compile(
    optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.001)
)

# **Tensorflow Graph concept - Relevant method for GAN - @tf.function: GradientTape**

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)

# save checkpoints
checkpoint_dir = "training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                discriminator_optimizer=discriminator_optimizer,
                                generator=generator,
                                discriminator=discriminator)

# Constant
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal((num_examples_to_generate, noise_dim))

# @tf.function enables to use compile with train_step
@tf.function
def train_step(images):
    noise = tf.random.normal((BATCH_SIZE, noise_dim))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)


    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

# Train function
def train(datasets, epochs):
    for epoch in range(epochs):
        start = time.time()

        for img_batch in datasets:
            train_step(img_batch)

        # Produce images for the GIF
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                epoch+1,
                                seed)

        # Save the model every 15 epochs
        if (epoch+1)%15 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)


        print("Time for epoch {} is {} sec".format(epoch+1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                                epoch,
                                seed)

# Predicion
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(8,8))

    for i in range(predictions.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow(predictions[i, :, :, :]*127.5 + 127.5)
        plt.axis('off')

    plt.savefig('generated_image/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

# Train
train(train_dataset, EPOCHS)

# Restore
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

# Plot
def display_image(epoch):
    return PIL.Image.open("generated_image/image_at_epoch_{:04d}.png".format(epoch))
display_image(EPOCHs)

In [ ]:
"""
We can use an other approach
"""
# @tf.function enables to use compile with train_step
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        # Train discriminator
        noise = tf.random.normal((BATCH_SIZE, noise_dim))
        generated_images = generator.predict(noise)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        disc_loss = discriminator_loss(real_output, fake_output)

        # Train generator
        noise = tf.random.normal((BATCH_SIZE, noise_dim))
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=False)
        gen_loss = generator_loss(fake_output)



    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))



# **Method using train_on_batch**

In [ ]:
"""
This method use an other approach for training : train_on_batch
https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-a-cifar-10-small-object-photographs-from-scratch/
"""

# Define the standalone discriminator model
def define_discriminator(in_shape=(32, 32, 3)):
    in_image = Input(shape=in_shape)
    # Normal
    x = Conv2D(64, (3, 3), padding='same')(in_image)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Downsample
    x = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    # Classifier
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(in_image, x)

    # Compile model
    opt = Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

# Define the standalone generator model
def define_generator(latent_dim):
    in_lat = Input(shape=(latent_dim,))
    # Foundation for 4x4 image
    n_nodes = 256 * 4 * 4
    x = Dense(n_nodes)(in_lat)
    x = LeakyReLU(alpha=0.2)(x)
    x = Reshape((4, 4, 256))(x)

    # Upsample to 8x8
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Upsample to 16x16
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Upsample to 32x32
    x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)

    # Output layer
    x = Conv2D(3, (3, 3), activation='tanh', padding='same')(x)

    model = Model(in_lat, x)

    return model

In [ ]:
"""
Define the gan model that combine discriminator that is frozen and generator that is not frozen
NB: Freeze discriminator
    Not Freeze generator
"""
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(learning_rate=0.0002, beta_1=0, beta_2=0.9)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model


In [ ]:
# load and prepare cifar10 training images
def load_real_samples():
    # load cifar10 dataset
    (trainX, _), (_, _) = load_data()
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X


# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y


# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input, verbose=0)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

In [ ]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
     # scale from [-1,1] to [0,1]
     examples = (examples + 1) / 2.0
     # plot images
     for i in range(n * n):
         # define subplot
         pyplot.subplot(n, n, 1 + i)
         # turn off axis
         pyplot.axis('off')
         # plot raw pixel data
         pyplot.imshow(examples[i])
     # save plot to file
     filename = 'generated_plot_e%03d.png' % (epoch+1)
     pyplot.savefig(filename)
     pyplot.show()

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
     # prepare real samples
     X_real, y_real = generate_real_samples(dataset, n_samples)
     # evaluate discriminator on real examples
     _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
     # prepare fake examples
     x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
     # evaluate discriminator on fake examples
     _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
     # summarize discriminator performance
     print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
     # save plot
     save_plot(x_fake, epoch)
     # save the generator model tile file
     filename = 'generator_model_%03d.h5' % (epoch+1)
     g_model.save(filename)



In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
     bat_per_epo = int(dataset.shape[0] / n_batch)
     half_batch = int(n_batch / 2)
     # manually enumerate epochs
     for i in range(n_epochs):
     # enumerate batches over the training set
         for j in range(bat_per_epo):
            """
              Train the discriminator
              NB: The same discriminator and generator is used in all training
            """
             # get randomly selected 'real' samples
             X_real, y_real = generate_real_samples(dataset, half_batch)
             # update discriminator model weights
             d_loss1, _ = d_model.train_on_batch(X_real, y_real)
             # generate 'fake' examples
             X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
             # update discriminator model weights
             d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)


            """
              Train the generator
            """
             # prepare points in latent space as input for the generator
             X_gan = generate_latent_points(latent_dim, n_batch)
             # create inverted labels for the fake samples
             y_gan = ones((n_batch, 1))
             # update the generator via the discriminator's error
             g_loss = gan_model.train_on_batch(X_gan, y_gan)
             # summarize loss on this batch
         print('epoch %d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
         (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
         # evaluate the model performance, sometimes
         if (i+1) % 10 == 0:
             summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
"""
NB: d_model et g_model sont utilisés pour créer gan_model
    Il faut noter que d_model.train_on_batch va mettre à jour les poids de d_model dans gan_model (c'est le même d_model)
    aussi toutes les modifications de g_model vont répercuter sur le g_model dans gan_model donc c'est le même
    """

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

# **Machine Translation**

In [ ]:
"""
Dans cet exemple, on pose l'hypothèse que les données sont déjà représentées sous forme de séquence c'est à dire qu'on n'a
pas besoin de la couche d'Embedding. Dans le cas courant, où les données sont représentées sous forme de vecteur, il faut nécessairement un embedding
pour avoir chaque mot représenté en vecteur.
https://www.kaggle.com/code/akshat0007/machine-translation-english-to-french-rnn-lstm
"""

In [ ]:
# Encoder
input_encoder = Input(shape=(None, features))
encoder = LSTM(units=latent_dim, return_states=True)
output_encoder, state_h, state_c = encoder(input_encoder)

# Decoder
input_decoder = Input(shape=(None, features))
decoder = LSTM(units=latent_dim, return_states=True, return_sequences=True)
output_lstm, _, _ = decoder(input_decoder, initial_state=[state_h, state_c])
dense_layer = Dense(units=vocab_size, activation="softmax")
dense_output = dense_layer(output_lstm)

# Model
model = Model([input_encoder, input_encoder], dense_output)

# Compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]) # one hot in label

# Train
model.fit([encoder_data, decoder_data], y_decoder, epochs=100, batch_size=32)

In [ ]:
"""
After training, we build an encoder and decoder depending on the training model for prediction
"""

# Build Encoder
encoder = Model(input_encoder, [output_encoder, state_h, state_c])

# Build Decoder
input_decoder = Input(shape=(1, features))
input_h = Input(shape=(latent_dim, ))
input_c = Input(shape=(latent_dim, ))
output_lstm, state_h state_c = decoder(input_decoder, initial_state=[input_h, input_c])
output_decoder = dense_layer(output_lstm)

decoder = Model([input_decoder, input_h, input_c], [output_decoder, state_h, state_c])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

# **BahdanauAttention in LSTM using Hidden state as query**

In [ ]:
"""
ChatBot Introduction Colab: https://www.kaggle.com/code/alincijov/dialog-chatbot-using-bahdanau-attention
"""

In [ ]:
# Split the dataset for train and test
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

BUFFER_SIZE = len(input_tensor_train)  # Define the buffer size, typically the number of training examples.
BATCH_SIZE = 64  # Define the batch size for training data.
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE  # Calculate the number of steps per training epoch.
embedding_dim = 256  # Define the dimension of word embeddings.
units = 1024  # Define the number of units or neurons in a recurrent neural network (RNN) layer.
vocab_inp_size = len(inp_lang.word_index) + 1  # Calculate the size of the input vocabulary.
vocab_tar_size = len(targ_lang.word_index) + 1  # Calculate the size of the target vocabulary.

# Create a TensorFlow dataset from the input and target tensors, and shuffle it using the specified BUFFER_SIZE.
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)

# Batch the dataset into batches of BATCH_SIZE and drop any remaining examples that don't fit into a batch.
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

# Get an example input batch and an example target batch from the dataset.
example_input_batch, example_target_batch = next(iter(dataset))

# Print the shapes of the example input and target batches.
print("Example Input Batch Shape:", example_input_batch.shape)
print("Example Target Batch Shape:", example_target_batch.shape)

In [ ]:
"""
BahdanauAttention: prend en entrée le hidden_state de l'encoder considéré comme query, et la sortie de l'encoder considéré comme key et value
La sortie de la couche BahdanauAttention utilisée dans le décodeur et concaténer avec le mot en cours de traitement (son embedding)
"""

In [ ]:
# Define a custom class called "Encoder" that inherits from the tf.keras.Model class.
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()  # Call the constructor of the parent class.
        self.batch_sz = batch_sz  # Store the batch size as an instance variable.
        self.enc_units = enc_units  # Store the number of units in the GRU layer.

        # Create an embedding layer to convert input tokens into dense vectors.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # Create a GRU (Gated Recurrent Unit) layer with specified parameters.
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    # Define the forward pass for the encoder.
    def call(self, x, hidden):
        x = self.embedding(x)  # Pass the input through the embedding layer. --------------------------------(None, sequence, embedding_dim)
        output, state = self.gru(x, initial_state=hidden)  # Pass through the GRU. --------------------------(None, sequence, enc_units), (None, enc_units)
        return output, state  # Return the output sequence and final hidden state. --------------------------(None, sequence, enc_units), (None, enc_units)

    # Initialize the hidden state (typically with zeros).
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
"""
Q = Decoder hidden state (None, decoder_unit)
    at the begining, Q = Encoder hidden state

V = Encoder output (None, sequence, feature)

Q = tf.expand_dims(Q, axis=1) # (None, 1, decoder_unit)
EncQ = Dense1(Q) # (None, units)
EncV = Dense2(V) # (None, sequence, units)
sum = tf.nn.tanh(EncQ+EncV) # activation (None, sequence, units)

score = Dense3(sum) # (None, sequence, 1)
attention = tf.nn.softmax(score, axis=1) # (None, sequence, 1) softmax based on token position

context_vector = attention*V # (None, sequence, feature)
context_vector = tf.nn.sum(context_vector, axis=1) # (None, feature)

context_vector sera utilisé dans le decoder et concaténer avec l'entrée (qui se fait token par token)
x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) # (None, 1, feature+embedding_dim_x) x vient de l'embedding x est de type (None, 1) car on traite par token Embedding donne (None, 1, embedding_dim_x)
"""

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        """
          query = hidden state dec = (None, latent_dim)
          values = outputs of enc = (None, sequence, latent_dim)
        """
        # (None, 1, latent_dim)
        query_with_time_axis = tf.expand_dims(query, 1)

        # (None, sequence, 1)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # (batch_size, sequence, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # (None, sequence, latent_dim)
        context_vector = attention_weights * values

        # (None, latent_dim)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0)) # return False if 0 and True else (0 is the pad token)
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask # reduce to 0 if correspond to pad token (False)

    return tf.reduce_mean(loss_)

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        # (BATCH_SIZE, 1)
        dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
EPOCHS = 40

# Training taking batch per batch
for epoch in range(1, EPOCHS + 1):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    if(epoch % 4 == 0):
        print('Epoch:{:3d} Loss:{:.4f}'.format(epoch,
                                          total_loss / steps_per_epoch))

# **tf.data.Dataset**

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_generator
# + load image: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

import tensorflow as tf

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class CustomDataGenerator(Sequence):
    def __init__(self, x_data, y_data, batch_size, shuffle=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.x_data))

        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return int(np.ceil(len(self.x_data) / self.batch_size)) # np.ceil : partie entière inférieure -->

    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size

        batch_x = self.x_data[self.indexes[start:end]]
        batch_y = self.y_data[self.indexes[start:end]]
        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Example usage:
# Create some example data
x_data = np.random.rand(100, 32, 32, 3)
y_data = np.random.randint(0, 2, size=(100,))

# Create an instance of the custom data generator
batch_size = 32
data_generator = CustomDataGenerator(x_data, y_data, batch_size)

# Iterate through the data generator for training
for epoch in range(epochs):
    for batch_x, batch_y in data_generator:
        # Train your model on batch_x and batch_y


**Source dataset**

In [ ]:
# Data don't fit in memory
data = [1,2,3]
dataset = tf.data.Dataset.from_tensor_slices(data)
for element in dataset:
    print(element)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [ ]:
# Read lines from Files
files = ["text1", "text2"]
dataset = tf.data.TextLineDataset(files)
for element in dataset:
    print(element)

tf.Tensor(b'Le senegal', shape=(), dtype=string)
tf.Tensor(b'La gambie', shape=(), dtype=string)
tf.Tensor(b'Le Maroc', shape=(), dtype=string)
tf.Tensor(b'La Tunisie', shape=(), dtype=string)


In [ ]:
# Read name of files giving extension
path = '*.txt'
dataset = tf.data.Dataset.list_files(path)
for element in dataset:
    print(element)

tf.Tensor(b'./text1.txt', shape=(), dtype=string)
tf.Tensor(b'./text2.txt', shape=(), dtype=string)


**Transformation**

In [ ]:
# map
map(
    map_func,
    num_parallel_calls=None,  # epresenting the number elements to process asynchronously in parallel. If not specified, elements will be processed sequentially. If the value tf.data.AUTOTUNE is used, then the number of parallel calls is set dynamically based on available CPU.
    deterministic=None,
    name=None
)

In [ ]:
# 1. map
dataset = tf.data.Dataset.from_tensor_slices([1,2,3])
dataset = dataset.map(lambda x: x**2)
list(dataset.as_numpy_iterator())

[1, 4, 9]

In [ ]:
# 2. map
dataset = tf.data.Dataset.from_tensor_slices(([1,2,3], [0,1,1]))
dataset = dataset.map(lambda x1, x2: (x1**2, x2))
list(dataset.as_numpy_iterator())

[(1, 0), (4, 1), (9, 1)]

In [ ]:
# filter
dataset = tf.data.Dataset.range(100)
dataset = dataset.filter(lambda x: x < 5)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [ ]:
# apply
dataset = tf.data.Dataset.range(100)

def my_filter(ds):
    return ds.filter(lambda x: x < 5)

dataset = dataset.apply(my_filter)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [ ]:
# as_numpy_iterator
dataset = tf.data.Dataset.from_tensor_slices({'a': ([1, 2], [3, 4]),
                                              'b': [5, 6]})
list(dataset.as_numpy_iterator()) == [{'a': (1, 3), 'b': 5},
                                      {'a': (2, 4), 'b': 6}]


True

In [ ]:
# batch
batch(
    batch_size,
    drop_remainder=False, # representing whether the last batch should be dropped in the case it has fewer than batch_size elements; the default behavior is not to drop the smaller batch.
    num_parallel_calls=None, # number of batches to compute asynchronously in parallel
    deterministic=None,
    name=None
)

In [ ]:
# batch
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3)
list(dataset.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7])]

In [ ]:
dataset = tf.data.Dataset.range(8)
dataset = dataset.batch(3, drop_remainder=True)
list(dataset.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5])]

In [ ]:
# cardinality
dataset = tf.data.Dataset.range(42)
print(dataset.cardinality().numpy())

42


In [ ]:
# flat_map
dataset = tf.data.Dataset.from_tensor_slices(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]])
dataset = dataset.flat_map(tf.data.Dataset.from_tensor_slices)
list(dataset.as_numpy_iterator())

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
# from_tensors: Creates a Dataset with a single element, comprising the given tensors.
dataset = tf.data.Dataset.from_tensors([1, 2, 3])
for element in dataset:
    print(element)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [ ]:
# from_tensors: Creates a Dataset with a single element, comprising the given tensors.
dataset = tf.data.Dataset.from_tensors([1, 2, 3])
list(dataset.as_numpy_iterator())

[array([1, 2, 3], dtype=int32)]

**Make dataset**

In [ ]:
data = tf.data.Dataset.range(10)
dataset = (
    data
    .map(lambda x: x**2, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(2)
)

for element in dataset:
  print(element)

tf.Tensor([0 1], shape=(2,), dtype=int64)
tf.Tensor([4 9], shape=(2,), dtype=int64)
tf.Tensor([16 25], shape=(2,), dtype=int64)
tf.Tensor([36 49], shape=(2,), dtype=int64)
tf.Tensor([64 81], shape=(2,), dtype=int64)


In [ ]:
# Slicing a 1D tensor produces scalar tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
list(dataset.as_numpy_iterator())

[1, 2, 3]

In [ ]:
# Slicing a 2D tensor produces 1D tensor elements.
dataset = tf.data.Dataset.from_tensor_slices([[1, 2], [3, 4]])
list(dataset.as_numpy_iterator())

[array([1, 2], dtype=int32), array([3, 4], dtype=int32)]

In [ ]:
# Slicing a tuple of 1D tensors produces tuple elements containing WAY FOR DATASET PROCESSING
# scalar tensors.
dataset = tf.data.Dataset.from_tensor_slices(([1, 2], [3, 4], [5, 6]))
list(dataset.as_numpy_iterator())

[(1, 3, 5), (2, 4, 6)]

In [ ]:
# Dictionary structure is also preserved.
dataset = tf.data.Dataset.from_tensor_slices({"a": [1, 2], "b": [3, 4]})
list(dataset.as_numpy_iterator())


[{'a': 1, 'b': 3}, {'a': 2, 'b': 4}]

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], [2, 1], [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
list(dataset.as_numpy_iterator())

[(array([1, 3], dtype=int32), b'A'),
 (array([2, 1], dtype=int32), b'B'),
 (array([3, 3], dtype=int32), b'A')]

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], [2, 1], [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
# Both the features and the labels tensors can be converted
# to a Dataset object separately and combined after.
features_dataset = tf.data.Dataset.from_tensor_slices(features)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
dataset = tf.data.Dataset.zip((features_dataset, labels_dataset))
list(dataset.as_numpy_iterator())

[(array([1, 3], dtype=int32), b'A'),
 (array([2, 1], dtype=int32), b'B'),
 (array([3, 3], dtype=int32), b'A')]

In [ ]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], [2, 3]],
                                [[2, 1], [1, 2]],
                                [[3, 3], [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2, 1))
dataset = tf.data.Dataset.from_tensor_slices((batched_features, batched_labels))
for element in dataset.as_numpy_iterator():
    print(element)
# show shape
print(dataset)

(array([[1, 3],
       [2, 3]], dtype=int32), array([[b'A'],
       [b'A']], dtype=object))
(array([[2, 1],
       [1, 2]], dtype=int32), array([[b'B'],
       [b'B']], dtype=object))
(array([[3, 3],
       [3, 2]], dtype=int32), array([[b'A'],
       [b'B']], dtype=object))
<_TensorSliceDataset element_spec=(TensorSpec(shape=(2, 2), dtype=tf.int32, name=None), TensorSpec(shape=(2, 1), dtype=tf.string, name=None))>


In [ ]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], [2, 3]],
                                [[2, 1], [1, 2]],
                                [[3, 3], [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2)) # change shape
dataset = tf.data.Dataset.from_tensor_slices((batched_features, batched_labels))
for element in dataset.as_numpy_iterator():
    print(element)
# show shape
print(dataset)

(array([[1, 3],
       [2, 3]], dtype=int32), array([b'A', b'A'], dtype=object))
(array([[2, 1],
       [1, 2]], dtype=int32), array([b'B', b'B'], dtype=object))
(array([[3, 3],
       [3, 2]], dtype=int32), array([b'A', b'B'], dtype=object))
<_TensorSliceDataset element_spec=(TensorSpec(shape=(2, 2), dtype=tf.int32, name=None), TensorSpec(shape=(2,), dtype=tf.string, name=None))>


**prefetch**

Creates a Dataset that prefetches elements from this dataset.

Most dataset input pipelines should end with a call to prefetch. This allows later elements to be prepared while the current element is being processed. This often improves latency and throughput, at the cost of using additional memory to store prefetched elements.

Note: Like other Dataset methods, prefetch operates on the elements of the input dataset. It has no concept of examples vs. batches. examples.prefetch(2) will prefetch two elements (2 examples), while examples.batch(20).prefetch(2) will prefetch 2 elements (2 batches, of 20 examples each).

In [ ]:
prefetch(
    buffer_size, # 	A tf.int64 scalar tf.Tensor, representing the maximum number of elements that will be buffered when prefetching. If the value tf.data.AUTOTUNE is used, then the buffer size is dynamically tuned.
    name=None
)

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.prefetch(2)
list(dataset.as_numpy_iterator())

[0, 1, 2]

**shuffle**

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=False)
dataset = dataset.repeat(2)
list(dataset.as_numpy_iterator())

[0, 2, 1, 0, 2, 1]

In [ ]:
dataset = tf.data.Dataset.range(3)
dataset = dataset.shuffle(3, reshuffle_each_iteration=True)
dataset = dataset.repeat(2)
list(dataset.as_numpy_iterator())

[2, 0, 1, 0, 2, 1]

**take**

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.take(3)
list(dataset.as_numpy_iterator())


[0, 1, 2]

**take_while**

In [ ]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.take_while(lambda x: x < 5)
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

**unbatch**

Splits elements of a dataset into multiple elements.

For example, if elements of the dataset are shaped [B, a0, a1, ...], where B may vary for each input element, then for each element in the dataset, the unbatched dataset will contain B consecutive elements of shape [a0, a1, ...].

In [ ]:
elements = [ [1, 2, 3], [1, 2], [1, 2, 3, 4] ]
dataset = tf.data.Dataset.from_generator(lambda: elements, tf.int64)
dataset = dataset.unbatch()
list(dataset.as_numpy_iterator())

[1, 2, 3, 1, 2, 1, 2, 3, 4]

**unique**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 37, 2, 37, 2, 1])
dataset = dataset.unique()
sorted(list(dataset.as_numpy_iterator()))

[1, 2, 37]

**window**

Returns a dataset of "windows".

Each "window" is a dataset that contains a subset of elements of the input dataset. These are finite datasets of size size (or possibly fewer if there are not enough input elements to fill the window and drop_remainder evaluates to False).



In [ ]:
dataset = tf.data.Dataset.range(7).window(3,
                                          drop_remainder=True)
for window in dataset:
  print(list(window.as_numpy_iterator()))

[0, 1, 2]
[3, 4, 5]


The shift argument determines the number of input elements to shift between the start of each window. If windows and elements are both numbered starting at 0, the first element in window k will be element k * shift of the input dataset. In particular, the first element of the first window will always be the first element of the input dataset.

In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1,
                                          drop_remainder=True)
for window in dataset:
    print(list(window.as_numpy_iterator()))

[0, 1, 2]
[1, 2, 3]
[2, 3, 4]
[3, 4, 5]
[4, 5, 6]


The stride argument determines the stride between input elements within a window.

In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1, stride=2,
                                          drop_remainder=True)
for window in dataset:
  print(list(window.as_numpy_iterator()))

[0, 2, 4]
[1, 3, 5]
[2, 4, 6]


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(([1, 2, 3, 4, 5],
                                              [6, 7, 8, 9, 10]))
dataset = dataset.window(2)
print(dataset)

<_WindowDataset element_spec=(DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])), DatasetSpec(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorShape([])))>


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(([1, 2, 3, 4, 5],
                                              [6, 7, 8, 9, 10]))
dataset = dataset.window(2)

for w in dataset: # w is a tuple
  print("#############")
  print(w[0].batch(2))
  print(list(w[0].as_numpy_iterator()))
  print(list(w[1].as_numpy_iterator()))

#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[1, 2]
[6, 7]
#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[3, 4]
[8, 9]
#############
<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>
[5]
[10]


In [ ]:
dataset = tf.data.Dataset.range(7).window(3, shift=1,
                                          drop_remainder=True)
batched = dataset.flat_map(lambda x:x.batch(3))
for batch in batched:
  print(batch)
  print(batch.numpy())

tf.Tensor([0 1 2], shape=(3,), dtype=int64)
[0 1 2]
tf.Tensor([1 2 3], shape=(3,), dtype=int64)
[1 2 3]
tf.Tensor([2 3 4], shape=(3,), dtype=int64)
[2 3 4]
tf.Tensor([3 4 5], shape=(3,), dtype=int64)
[3 4 5]
tf.Tensor([4 5 6], shape=(3,), dtype=int64)
[4 5 6]


**zip**

In [ ]:
a = tf.data.Dataset.range(1, 4)  # ==> [ 1, 2, 3 ]
b = tf.data.Dataset.range(4, 7)  # ==> [ 4, 5, 6 ]
ds = tf.data.Dataset.zip(a, b)
list(ds.as_numpy_iterator())

[(1, 4), (2, 5), (3, 6)]

In [ ]:
ds = tf.data.Dataset.zip(b, a)
list(ds.as_numpy_iterator())

[(4, 1), (5, 2), (6, 3)]

In [ ]:
# The `datasets` argument may contain an arbitrary number of datasets.
c = tf.data.Dataset.range(7, 13).batch(2)  # ==> [ [7, 8],
                                           #       [9, 10],
                                           #       [11, 12] ]
ds = tf.data.Dataset.zip(a, b, c)
for element in ds.as_numpy_iterator():
  print(element)

(1, 4, array([7, 8]))
(2, 5, array([ 9, 10]))
(3, 6, array([11, 12]))


In [ ]:
# The number of elements in the resulting dataset is the same as
# the size of the smallest dataset in `datasets`.
d = tf.data.Dataset.range(13, 15)  # ==> [ 13, 14 ]
ds = tf.data.Dataset.zip(a, d)
list(ds.as_numpy_iterator())

[(1, 13), (2, 14)]

In [ ]:
a = tf.data.Dataset.from_tensor_slices(([1,2,3], [5,5,7]))
for i in a:
  print(i)

(<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>)
(<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=7>)


# **Image - Text - Sequence**

* **Module**

* **Class**

* **Function**

In [ ]:
"""
Sur Tensorflow, on a des modules, des classes et des fonctions
tf.keras.preprocessing: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing [Module: image, text, sequence]
tf.keras.utils: https://www.tensorflow.org/api_docs/python/tf/keras/utils: exemple :
tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)
"""

In [ ]:
# Fine-tune model application
https://keras.io/api/applications/

# **Callbacks**

In [ ]:
"""
callbacks permet de controler le train
tf.keras.callbacks.Callback
https://www.tensorflow.org/api_docs/python/tf/keras/callbacks [Callback, ModelCheckpoint]
"""

In [ ]:
import tensorflow as tf
from tensorflow import keras
"""
self c'est le modèle
"""
class MyCustomCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Starting epoch {epoch}")

    def on_epoch_end(self, epoch, logs=None):
        print(f"Finished epoch {epoch}. Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}")

    def on_train_batch_begin(self, batch, logs=None):
        print(f"Training batch {batch}...")

    def on_train_batch_end(self, batch, logs=None):
        print(f"Finished training batch {batch}. Loss: {logs['loss']:.4f}")

    def on_test_begin(self, logs=None):
        print("Starting validation...")

    def on_test_end(self, logs=None):
        print("Finished validation. Loss: {logs['loss']:.4f}, Accuracy: {logs['accuracy']:.4f}")

    def on_predict_begin(self, logs=None):
        print("Starting prediction...")

    def on_predict_end(self, logs=None):
        print("Finished prediction.")

# Créez un modèle Keras simple à des fins d'exemple
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(784,)),
    keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Générez des données factices pour l'exemple
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Entraînez le modèle avec l'utilisation de votre callback personnalisé
history = model.fit(x_train, y_train, epochs=3, callbacks=[MyCustomCallback()])

# Effectuez une évaluation du modèle
model.evaluate(x_test, y_test, callbacks=[MyCustomCallback()])

# Effectuez des prédictions
model.predict(x_test[:5], callbacks=[MyCustomCallback()])


In [ ]:
checkpoint_dir = 'drive/MyDrive/autoACN_densenet40_cifar100'
# Créer un callback pour enregistrer les checkpoints tous les 50 époques
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_dir, 'model_weights_{epoch:03d}.h5'),
    save_weights_only=True,
    save_best_only=False,
    save_freq=SAVE_PERIOD*STEPS_PER_EPOCH # Sauvegarder tous les 50 époques
)
# Train the DenseNet-40 model
history = model_densenet40.fit(train_gen,
                     steps_per_epoch=x_train.shape[0] // batch_size, epochs=epochs,
                     validation_data=(x_val, y_val), callbacks=[keras.callbacks.LearningRateScheduler(lr_schedule), checkpoint_callback])


In [ ]:
print('Restored model, accuracy: {:5.2f}%'.format(100 * 2.08))

Restored model, accuracy: 208.00%


In [ ]:
value = 10*3.09
a = '{:.05f}'.format(value)
a

'30.90000'

In [ ]:
value = 10*3.09
a = f'{value:.02f}'
a

'30.90'

In [ ]:
value = 1009*3.09
a = f'{value:19.02f}'
a

'            3117.81'

In [ ]:
a = '{:2d}'.format(10)
a

'10'

In [ ]:
a = '{:03d}'.format(10)
a

'010'

In [ ]:
"Value: %03d" % 2

'Value: 002'

In [ ]:
"Value: %.03f" % 2

'Value: 2.000'

In [ ]:
"Value: %.03f %02d" % (2, 2)

'Value: 2.000 02'

# **Save Model**

In [ ]:
https://www.tensorflow.org/tutorials/keras/save_and_load

Les formats de fichiers .keras, .ckpt (Checkpoint), et .h5 (HDF5) sont tous utilisés pour enregistrer des modèles TensorFlow (notamment avec l'API Keras). Chacun a ses propres caractéristiques et avantages. Voici les principales différences entre ces formats de fichiers :

**.keras (SavedModel):**

Format par défaut lors de l'enregistrement d'un modèle Keras avec TensorFlow.
Contient le modèle complet, y compris l'architecture, les poids, la configuration et les informations d'entraînement.
Peut être utilisé pour le déploiement en production et pour la reprise de l'entraînement sans avoir à répéter la définition du modèle.
Fournit une structure hiérarchique qui permet de stocker plusieurs versions du modèle et des signatures de fonction pour le déploiement.<br>
**Le modèle est enrégistrer sur un fichier zip. model.save("model.keras")**<br>

**.ckpt (Checkpoint):**

Enregistre principalement les poids du modèle, pas l'architecture.
Utilisé pour sauvegarder uniquement les poids d'un modèle, ce qui permet de réutiliser les mêmes couches avec des architectures différentes.
Souvent utilisé pour la reprise de l'entraînement et la sauvegarde de points de contrôle (checkpoints) à différents stades de l'entraînement.
Vous devez spécifier l'architecture du modèle lors du chargement des poids depuis un fichier .ckpt.<br>
**Enrégistre trois fichiers. model.save_weights("model.ckpt")**<br>


**.h5 (HDF5):**

Peut enregistrer l'ensemble du modèle, y compris l'architecture, les poids et la configuration.
Facile à partager avec d'autres utilisateurs, car il contient toutes les informations nécessaires pour reconstruire et utiliser le modèle.
Utile pour enregistrer des modèles complets pour la reprise de l'entraînement ou pour des expériences de recherche.
Peut également être utilisé pour enregistrer uniquement les poids si vous le souhaitez.
En résumé, le format .keras (SavedModel) est généralement recommandé pour le déploiement en production, car il contient toutes les informations nécessaires pour le modèle, tandis que le format .ckpt est idéal pour la reprise de l'entraînement et la sauvegarde des poids. Le format .h5 est polyvalent, car il peut enregistrer l'ensemble du modèle ou uniquement les poids, ce qui le rend pratique pour diverses situations. Le choix du format dépend de l'utilisation spécifique de votre modèle et de vos besoins en matière de partage et de reprise de l'entraînement.<br>
**Enrégistre un seul fichier avec model.save("model.h5") ou model.save_weights("model.h5")**